In [43]:
import numpy as np
import os,sys

sys.path.append('../RL_lib/Utils')
sys.path.append('../Asteroid_shapemodels')
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/briangaudet/Study/Subjects/MachineLearning/Projects/Asteroid_CPO/Lidar_models


In [11]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [13]:
from lidar_measurement_model import Lidar_measurement_model
from altimeter_pointing import Altimeter
sm = 1000*np.load('../Asteroid_shapemodels/downsampled_rq36_01.npy')
print('Shape Model: ', np.min(sm,axis=0), np.max(sm,axis=0), sm.shape[0])
#sm = 1000*np.load('rq36.npy')
mm = Lidar_measurement_model(sm)
az = np.max(sm[:,2])
print(az)

Shape Model:  [-272.726875   -281.00798611 -266.34577465] [276.28392593 283.73243902 287.18      ] 12980
Half Shape Model:  [-2.68968942e+02 -2.77535823e+02  1.00000000e-02] [275.44661871 283.73243902 287.18      ] 6516
Shape Model Vertex Spacing (mean / min / max):  6.842243150812332 0.7947696067482021 10.219420066768421
287.18


In [41]:
from time import time
target = np.asarray([0.,0.,287.])
am = Altimeter(mm, target,theta=np.pi/16)
t0 = time()
p_range, p_dopplar = am.get_reading(np.asarray([0,200,1000]) ,np.asarray([-0.1,0.1,-1.0]))
t1 = time()
print(t1-t0)
print(p_range,p_dopplar)


0.0047969818115234375
[ 1288.35415033  1135.1985074   1304.8729501  10000.        ] [0.98342525 1.11610436 0.97097576 0.        ]


In [42]:
from time import time
target = np.asarray([0.,0.,287.])
am = Altimeter(mm, target,theta=np.pi/160)
t0 = time()
p_range, p_dopplar = am.get_reading(np.asarray([0,0,1000]) ,np.asarray([-0.1,0.1,-1.0]))
t1 = time()
print(t1-t0)
print(p_range,p_dopplar)


0.005096912384033203
[1007.05333401 1003.48147441 1003.98472592 1008.76768931] [1.27798594 1.28253489 1.28189201 1.27581406]


In [36]:
m = 1000
sm = mm.shape_model 
err1 = []

vels = []
ranges = []
for i in range(m): 
    #print('******* ', i)
    pos = np.random.uniform(low=[-50,-50,300],high=[50,50,500])
    #print(pos)
    vel = np.random.uniform(low=[-0.01,-0.01,-0.1],high=[0.01,0.01,0.1])
    idx = np.random.randint(0,sm.shape[0])
    vertex = sm[idx]
    r = pos-vertex 
    rdvec = -r / np.linalg.norm(r)
    gt_range = np.linalg.norm(pos-vertex)
    p_range, p_dopplar, _ = mm.get_range(pos + 1e-1,vel,rdvec)
    err1.append(p_range-gt_range)
    vels.append(p_dopplar)
    ranges.append(p_range)
    #print('closest: ',closest)
    #print('vertex: ', vertex)
    #print('rdvec: ', rdvec)
    #print(p_range-gt_range)
    
print(np.mean(np.abs(err1)),np.max(np.abs(err1)))
print(np.min(vels), np.max(vels))
print(np.min(ranges), np.max(ranges))

0.49712387286318177 104.27725286665037
-0.4192247918064912 0.4259918591544926
63.277548809424985 591.9006244054923
